In [ ]:
import sqlalchemy as sa
import pandas as pd
import os


# add run method to engine
def run(self: sa.engine.Engine, sql: str) -> pd.DataFrame | None:
    with self.begin() as conn:
        res = conn.execute(sa.text(sql))
        if res.returns_rows:
            return pd.DataFrame(res.all(), columns=res.keys())


sa.engine.Engine.run = run

### pg

In [ ]:
connection_string = f"://:{os.environ['password']}@"
eng = sa.create_engine(connection_string)

In [ ]:
qry = f"""
SELECT 
    tc.table_schema,
    tc.table_name,
    kcu.column_name,
    tc.constraint_name,
    tc.constraint_type
FROM information_schema.table_constraints tc
JOIN information_schema.key_column_usage kcu 
    ON tc.constraint_name = kcu.constraint_name
    AND tc.table_schema = kcu.table_schema
WHERE tc.constraint_type = 'PRIMARY KEY'
    AND tc.table_schema = 'kraken'
    AND tc.table_name = 'trades_history'
ORDER BY kcu.ordinal_position;
"""

df = eng.run(qry)
df

### tera

In [ ]:
# Teradata query for table constraints and indexes
qry = f"""
SELECT 
    d.DatabaseName as table_schema,
    t.TableName as table_name,
    i.ColumnName as column_name,
    i.IndexName as constraint_name,
    CASE 
        WHEN i.UniqueFlag = 'Y' THEN 'UNIQUE'
        WHEN i.IndexType = 'P' THEN 'PRIMARY KEY'
        WHEN i.IndexType = 'S' THEN 'SECONDARY INDEX'
        WHEN i.IndexType = 'J' THEN 'JOIN INDEX'
        ELSE 'INDEX'
    END as constraint_type,
    i.IndexType,
    i.UniqueFlag,
    i.ColumnPosition
FROM DBC.IndicesV i
JOIN DBC.TablesV t ON i.DatabaseName = t.DatabaseName AND i.TableName = t.TableName
JOIN DBC.DatabasesV d ON i.DatabaseName = d.DatabaseName
WHERE i.DatabaseName = 'your_database_name'
    AND i.TableName = 'your_table_name'
    AND t.TableKind = 'T'  -- Only tables, not views
ORDER BY i.IndexName, i.ColumnPosition;
"""

df = eng.run(qry)
df